<a href="https://colab.research.google.com/github/MinsooKwak/Kaggle_note/blob/main/Data_science/porto_seguro's_safe_driver_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필사 스터디 참고: 

https://www.kaggle.com/code/gpreda/porto-seguro-exploratory-analysis-and-prediction/notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 무작위 순열 # 무작위 셔플 넣기
from sklearn.utils import shuffle # 일관된 방식으로 배열 또는 희소 행렬을 섞는다
from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# VarianceThreshold : 변수의 variance가 threshold보다 작으면 drop
from sklearn.feature_selection import VarianceThreshold
# SelectFromModel은 지도학습 모델로 계산된 중요도가 지정한 임계치보다 큰 모든 특성을 선택
# 단변량과는 달리 한 번에 모든 특성을 고려해 상호작용 부분을 적용 가능
from sklearn.feature_selection import SelectFromModel

# 교차검증 : 과적합을 방지하기 위한 방법
# StratifiedKFold는 각 클래스에 대한 비율을 고려해 fold를 나눔
from sklearn.model_selection import StratifiedKFold
# 교차검증을 더 쉽게 하기 위한 API
from sklearn.model_selection import cross_val_score

In [4]:
#!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_columns',100)

### Imputation 방법론 (study)

- Drop 
  - 중요한 정보를 가진 데이터를 잃을 위험 있음
  - 통계적 편향이 생김

  => Impuation

**[Single Impuation]**
- Mean, Median Imputation
  - 작은 데이터셋에 용이
  - 숫자형 데이터에만 사용할 수 있음
  - 결측치가 존재하는 컬럼만 고려
  - 다른 feature 간의 상관관계가 고려되지 않음

    ```
    from sklearn.impute import SimpleImputer

    imp_mean = SimpleImputer(strategy = 'mean') 
    # median 쓰면 중앙값
    imp_mean.fit(df)
    ```
    - Mean imputation
    - Median Imputation

- Most Frequent Value Imputation
  - 가장 빈번히 나온 값으로 대체할 수 있음
  - 범주형 feature에도 활용할 수 있음

  ```
  from skelarn.impute import SimpleImputer
  imp_mean =SimpleImputer(stretegy='most_frequent')
  imp_mean.fit(df)
  df_imputed = pd.DataFrame(imp_mean.transform(df))
  ```

- Zero Imputation / Constant Imputation
  - 0으로 대체 / 지정한 상수값으로 대체
  - 범주형 feature에 잘 작동함
  - 다른 feature 간의 상관관계가 고려되지 않음
  - 데이터에 bias(편향)을 만들 수 있음


- K-NN Imputation
  - feature similarity를 활용하여 가장 닮은(근접한) 데이터를 K개 찾는 방식
  - impyute라이브러리를 활용
  - KDTree를 생성한 후 가장 가까운 이웃(NN) 찾음
  - K개의 NN 찾은 후 거리에 따른 가중 평균 취함
  - mean, median, most_frequent 보다 정확
  - 메모리가 많이 필요하며, 전체 데이터셋을 메모리에 올려야한다.
  - 이상치에 민감하다.

  ```
  from impyute.imputation.cs import fast_knn
  np_imputed = fast_knn(df.values, k=5) # knn 학습
  df_imputed = pd.DataFrame(np_imputed)
  ```


- Stochastic regression imputation
  - 동일 데이터셋, 관련 다른 feature에서 missing value 예측
  - 다른 값들로 회귀 추정을 해서 imputation 함
  - 신뢰도가 과대평가 되는 것을 방지하기 위해 residual(잔차)를 추가해줌
  - simple impuation 중엔 가장 편향이 적은 결과(여전히 과대평가)

- Extrapolation and Interpolation
  - 이산형 범위 내 다른 데이터로 부터 값 추정
  - interpolation
    - 20살일때의 키, 40살일 때의 키 -> 30살일때의 키
  - extrapolation
    - 1살~현재 키 -> 앞으로 10년 후의 키 예측
    - interpolation보다 안정성 떨어짐

- Hot-Deck Imputation
  - 관련된 비슷한 데이터셋에서 랜덤하게 선택
  - 가장 많은 것을 넣거나, 안변했을 것이다

**[Multiple Impuation]**

=> 결측 대체값들을 만들 때마다 residual로 인한 변동으로 관측값보다 상대적으로 약한 계수 추정 신뢰도를 갖게 됨

=> single impuation을 거친 여러개의 데이터셋을 만들어 평가함 


- MICE(Multivariate Impustation by Chained Equation) Imputation
  - 누락된 데이터를 여러 번 채움
  - 불확실성을 고려하면 Single Imputation보다 나음
  - chained equation은 유연하여 연속형, 이진형, 범위형, survey skip 패턴을 처리할 수 있음
  - impuation, Analysis, Pooling
  - distribution을 토대로 m개의 데이터셋을 imputation 함
  - m개의 완성된 데이터셋 분석
  - 평균, 분산, 신뢰구간을 계산해 결과를 합침

  ```
  from impyute.impuation.cs import mice
  np_imputed = mice(df.values) # mice 학습 시작
  df_imputed = pd.DataFrame(np_imputed)
  ```

- Deeplearning 이용한 Imputation / Datawig
  - 범주형/ non numerical feature에 효과적
  - DNN 통해 학습하고 누락된 값 유추
  - 타 방식에 비해 정확
  - CPU, GPU 지원
  - 한 번에 한개의 컬럼만 대체 가능
  - 대규모 데이터셋에 속도가 느림
  - 유추하는 feature에 관련한 정보가 들어있는 feature 직접 지정해야 함


  ```
  import datawig
  imputer = datawig.SimpleImputer(
    input_columns = ['1','2','4'] # '3'컬럼 제외
    output_column = '0' # 컬럼 0의 결측치 채움
    imputer.fit(train_df=df, num_epochs=50)
    df_null = df[df['0'].isnull()]
    np_imputed = imputer.predict(df_null)
    df_imputed = pd.DataFrame(np_imputed)
  ```



참고1: https://dining-developer.tistory.com/19

참고2: https://m.blog.naver.com/hancury/220396495672

### 다항회귀

- 데이터들간의 형태가 비선형일 때 현재 데이터를 다항식 형태로 변경

  ```
  from sklearn.preprocessing import PolynomialFeatures

  poly_features= PolynominalFeatures(degree=2, include_bias=False)
  x_poly = poly_feautures.fit_transform(x)
  # parameter:
    # degree : 차수 조절
    # include_bias = True : 0차항도 함께 만듦
  ```

참고: https://inuplace.tistory.com/515

### Feature Selection

Feature Selection Method

1. Filter's method
- 단일 변수를 통계량을 이용해 평가
- 다른 method에 비해 계산량, 시간 적게 소모
- 다른 변수와의 상관관계를 고려하지 않으므로, 모델 성능이 떨어짐
- variance threshold, correlation coefficient
  - **variance threshold** : 변수의 variance가 threshold보다 작으면 drop 
    - 변수의 varience가 작으면 변수의 value가 target에 미치는 영향의 차이가 미비하기 떄문
    - 개별 변수의 varience만 고려함(단점)
    - 변수의 variance가 작더라도 target과의 상관관계는 높을 수 있으며 target에 영향을 줄 수 있지만 제거 (모델의 성능에 영향)

    ```
    from sklearn.feature_selection import VarienceThreshold
    selector = VarienceThreshold(0.8)
    train_thresh = selector.fit(train)
    ```
  - **correlation coefficient**
    - Target변수, 개별 feature 상관계수 구하고 상관계수 절댓값 큰 feature 선택


2. Wrappers' method
- 모델을 학습하며 feature 선택하는 과정
- Feature를 하나씩 제거하거나 더하며(모든 feature subset을 하나씩 학습하며) 가장 성능이 높게 나오는 feature set을 선택하는 방법
- Filter's method보다는 compuation, 시간 오래 걸림(단점)
- 모든 feature subset을 평가해 우수한 성능
- Recursive Feature Elimination(RFE)
  - 전체 feature를 포함하여 모델을 학습하고 이후 feature를 하나씩 제거하며 가장 성능이 좋은 feature subset을 결정하는 방법
  - 최종 선택 개수는 주관적이므로 RRECV를 통해 cross validation을 통해 적절한 feature 수 결정함
  
    ```
    rfe = RFE(rf, n_features_to_select=20)
    train_rfe = rfe.fit(X,y)
    # n_features_to_select : 최종적으로 선택할 feature의 수
    # get_support() : 어떤 feature가 선택되었는지 확인 가능
    ```
    - #RFECV 이용 방법:

    ```
    selector = RFECV(rf, step=1, cv=3)
    # step : 테스트 한 번 마다 몇개의 feature 제거할 것인지
    selector.fit(X,y)

    print(selector.support_) # 선택된 feature 확인 (True=선택, False=제거)
    print(selector.ranking_) # 각 feature의 ranking 확인 (ranking=1: 선택/ 그 이외 제외)
    ```




3. Embedded methods
- filter's methods + wrapper's methods
- tree 기반: feature importance
    - tree 기반 알고리즘을 사용하는 모델에 적용
    - information gain과 관련 있음
      - 상위 노드의 불순도-분기한 후 좌/우 노드의 불순도를 계산한 것
      - information gain이 크면 노드가 분기했을 때 불순도가 많이 감소한다는 뜻
    - 모델은 information gain이 큰 node를 선택해 그 node 기준으로 트리를 분기함
    - 불순도를 많이 감소시킬수록 importance가 커짐
- L1, L2 regulation
  - L1 규제 : 일부 특성의 계수만  학습(Lasso)



참고1: https://datascienceschool.net/03%20machine%20learning/14.03%20%ED%8A%B9%EC%A7%95%20%EC%84%A0%ED%83%9D.html

참고2: https://hyewonleess.github.io/ml/feature_selection/

SelectFromModel 참고 : https://woolulu.tistory.com/66

### 교차검증

한 번의 학습을 통해 평가를 하면 과적합을 할 가능성이 크다.

교차검증을 통해 과적합을 막아줄 수 있다.

1. 데이터가 독립적이고 동일한 분포
  - KFold, RepeatedKFold, LeaveOneOut(LOO), LeaveOutLeaveOneOut(LPO)
2. 데이터가 동일한 분포가 아닐 때
  - StratifiedKFold,RepeatedStratifiedKFold, StratifiedShuffleSplit
3. 그룹화된 데이터의 경우
  - GroupKFold, LeaveOneGroupOut, LeavePGroupsOut, GroupShuffleSplit
4. 시계열 데이터
  - TimeSeriesSplit

---

- **K-Fold** : k 값 만큼의 폴드 세트에 k번의 학습과 검증을 진행함
  - 일정한 간격으로 잘라서 사용
  - class imbalance를 고려하지 않음

  참고: https://continuous-development.tistory.com/166

- **StratifiedKFold** : 
  - 레이블 값의 분포를 반영
  - KFold과 같이 데이터가 한 곳으로 몰리는 것을 방지 (class imbalance 고려)

- 교차검증 쉽게 하기 위한 API : cross_val_score()
  - cross_val_score(estimator, X, y, scoring=평가지표, cv=교차 검증 폴드 수)
    - estimator == classifier종류
      - 내부적으로 stratifiedKFold 진행
    - cv 지정 횟수만큼 평가지표로 평가 결과값 배열로 반환
      - 평균하여 평가수치로 일반적으로 활용

참고: https://davinci-ai.tistory.com/18
